# Notebook Setup

As usual, we need to install Drake and our KinovaStation software, so go ahead and run this cell to do so. 

In [35]:
#@title Run Notebook Setup
import importlib
import sys
import os
from urllib.request import urlretrieve
import subprocess
import shutil

assert 'google.colab' in sys.modules, "This notebook is meant to be run in google colab!"

drake_url = "https://drake-packages.csail.mit.edu/tmp/drake-0.27.0-pip-bionic.tar.gz"
if importlib.util.find_spec('pydrake') is None:
    # We're in colab and don't have pydrake, so install it on the cloud machine.
    if os.path.isdir('/opt/drake'):
        shutil.rmtree('/opt/drake')
    print("Installing Drake")
    urlretrieve(drake_url, 'drake.tar.gz')
    subprocess.run(['mkdir', '/opt/drake'])
    subprocess.run(['tar', '-xzf', 'drake.tar.gz', '-C', '/opt/drake'], check=True)
    
    print("Installing other dependencies")
    subprocess.run(["pip3", "install", "meshcat"])
    subprocess.run(['apt-get', 'update', '-o', 'APT::Acquire::Retries=4', '-qq'], check=True)
    with open("/opt/drake/share/drake/setup/packages-bionic.txt", "r") as f:
        packages = f.read().splitlines()
    subprocess.run(['apt-get', 'install', '-o',
                    'APT::Acquire::Retries=4', '-o', 'Dpkg::Use-Pty=0',
                    '-qy', '--no-install-recommends'] + packages,
                    check=True)
    
    v = sys.version_info
    path = f"/opt/drake/lib/python{v.major}.{v.minor}/site-packages"
    if importlib.util.find_spec('pydrake') is None:
        sys.path.append(path)

# Start a meshcat server
print("Starting Meshcat")
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=['--ngrok_http_tunnel'])

# Clone our github repo
install_path = '/opt/kinova_drake'
if not os.path.isdir(install_path):
    print("Cloning github repo")
    subprocess.run(['git','clone','https://github.com/vincekurtz/kinova_drake.git',install_path])
sys.path.append(install_path)

# Install open3d point cloud library
print("Installing Open3D")
subprocess.run(['pip3','install','open3d'])

print("Done!")

Starting Meshcat
Installing Open3D
Done!


# All About System Diagrams

In the previous notebook, we saw how a KinovaStation object from [this repository](https://github.com/vincekurtz/kinova_drake) could be used to model the robot in simulation. Now we'll take a closer look at how this works. 

First, let's create a simple KinovaStation instance that we can play around with. 

In [ ]:
from kinova_station import KinovaStation
station = KinovaStation(time_step=0.002)
station.SetupSinglePegScenario()
station.ConnectToMeshcatVisualizer(zmq_url=zmq_url)
station.Finalize()

You can find out the type of any object in python using the `type` command. Let's do that for our station object. 

In [ ]:
print(type(station))

You can see that this is an instance of a `KinovaStation` class, which is defined [here](https://github.com/vincekurtz/kinova_drake/blob/master/kinova_station/simulation_station.py). Furthermore, we can see that this class inherits from a pydrake system diagram class.

(Need a refresher on classes, objects, and inheritance in python? Check out [this tutorial](https://realpython.com/python3-object-oriented-programming/) or find another one online. If you find a really good tutorial, send it to me! )

In [ ]:
print(type(station).__bases__)

This `Diagram` class is part of [Drake](https://drake.mit.edu/), the underlying software that we use for modeling and control in simulation and on the actual robot. You can read more about the Diagram class in the [Drake documentation](https://drake.mit.edu/pydrake/pydrake.systems.framework.html?#pydrake.systems.framework.Diagram), or in [this tutorial](https://hub.gke2.mybinder.org/user/robotlocomotion-drake-inrx89fz/notebooks/tutorials/dynamical_systems.ipynb) but we'll provide a quick overview here. 

Basically, a `Diagram` consists of a bunch of blocks with input and output ports. Each block represents some sort of subsystem, which takes the inputs, runs some computations, and sends outputs. 

You can view a visual representation of a system diagram using the `plot_system_graphviz` command. Let's give that a try with our `station`:


In [ ]:
from pydrake.all import plot_system_graphviz
from matplotlib import pyplot as plt
plt.figure(figsize=(20,20))
plot_system_graphviz(station)
plt.show()

Wow, that looks complicated! Don't worry, you don't need to know the details of all of those components. One of the nice thing about system diagrams is that they can be *abstracted*. That is, we can think of the station itself as a block in a larger diagram with certain inputs and outputs. Then we can just consider these inputs and outputs, without worrying about the details of what is going on inside the station. 

We can see just the inputs and outputs of the station by adjusting the `max_depth` parameter:

In [ ]:
plot_system_graphviz(station, max_depth=0)
plt.show()

As you can see, our station has four inputs (listed on the left side) and eight outputs (listed on the right side). In the next section, we'll give a brief summary of each of these. 

# Inputs and Outputs of the `KinovaStation`

## Outputs

Let's first consider the system outputs. These are things that we can measure using sensors on the real robot. 

The first three outputs have to do with all the joints of the robot:

1. `measured_arm_position` - Provides all the joint angles of the robot. Our robot has 7 degrees-of-freedom (i.e., it's 7-DoF robot), so there are 7 joint angles to consider. These are stacked together in a vector which we call $\mathbf{q}$:
\begin{equation*}
    \mathbf{q} = 
    \begin{bmatrix}
        q_1 \\
        q_2 \\
        \vdots \\
        q_7
    \end{bmatrix},
\end{equation*}
where $q_1$ is the angle (in radians) of the first joint, $q_2$ is the angle of the second joint, and so on. 

2. `measured_arm_velocity` - Provides all the joint velocities of the robot. Each joint velocity is the rate of change in the corresponding joint angle. These are stacked together in a similar vector:
\begin{equation*}
    \mathbf{v} = 
    \begin{bmatrix}
        v_1 \\
        v_2 \\
        \vdots \\
        v_7
    \end{bmatrix},
\end{equation*}
with $v_1$ corresponding to the first joint, $v_2$ corresponding to the second joint, and so on. If you're familiar with calculus, you may recognize that $\mathbf{v}$ is the derivative of $\mathbf{q}$, so we often write $\mathbf{v} = \dot{\mathbf{q}}$.

3. `measured_arm_torque` - Provides a vector of torques that are applied at each joint:
\begin{equation*}
    \boldsymbol{\tau} = 
    \begin{bmatrix}
        \tau_1 \\
        \tau_2 \\
        \vdots \\
        \tau_7
    \end{bmatrix}.
\end{equation*}

The next three outputs have to do with the *end-effector*. The end-effector is basically the "hand" position of the robot. In many cases, we care more about what the end-effector is doing than each of the individual joints. 

4. `measured_ee_pose` - The *pose* refers to position and orientation, in this case of the end-effector. This output port provides a vector of 6 elements. First the orientation is expressed as roll ($r$), pitch ($p$) and yaw ($w$), then the positoin is expressed in 3D cartesian space using the conventional coordinates ($x, y, z$). All of these elements are stacked in one pose vector:
\begin{equation*}
    \mathbf{p} = 
    \begin{bmatrix}
        r \\
        p \\
        w \\
        x \\
        y \\
        z
    \end{bmatrix}.
\end{equation*}

5. `measured_ee_twist` - The *twist* is merely the derivative (rate of change) of the pose. You can think of the first three elements as corresponding to the angular velocity of the end-effector, and the second three elements as the linear velocity of the end-effector. 
\begin{equation*}
    \dot{\mathbf{p}} = 
    \begin{bmatrix}
        \dot{r} \\
        \dot{p} \\
        \dot{w} \\
        \dot{x} \\
        \dot{y} \\
        \dot{z}
    \end{bmatrix}.
\end{equation*}

6. `measured_ee_wrench` - Finally we have the *wrench*, which is basically the force and torque applied at the end-effector. If $\mathbf{p}$ is analogous to $\mathbf{q}$ and $\dot{\mathbf{p}}$ is analogous to $\mathbf{v}$, The wrench $\mathbf{w}$ is analogous to $\boldsymbol{\tau}$. 

The final two outputs have to do with the gripper. 

7. `measured_gripper_position` - A single number between 0 and 1. A fully closed gripper corresponds to position 1, and a fully open gripper corresponds to position 0. 

8. `measured_gripper_velocity` - A single number, representing the derivative (rate of change) of the gripper position as defined above. Negative values open the gripper, while positive values close it. 

## Inputs

While there are technically four inputs to the `Kinovastation`, only two of them are particularly interesting (`ee_target` and `gripper_target`), while the others just determine what sort of command we're sending. 

1. `ee_target_type` - Determines what sort of end-effector commands we will send. These can be end-effector pose targets, end-effector twist targets, or end-effector wrench targets. We'll primarily use twist targets for now. 

2. `ee_target` - The command sent to the robot. So for example, if we send a twist type command, the robot will try to move so that the end-effector's linear and angular velocities match the command as well as posisble. 

3. `gripper_target_type` - Determines what sort of gripper command we'll send. We can send position or velocity commands. 

4. `gripper_target` - The position or velocity command to send to the gripper. 

# Creating a Custom Controller Block

Now let's see how we can use our knowledge of the `KinovaStation`'s inputs and outputs to write a custom controller. Specifically, our controller will send twist commands to the robot. 

In [41]:
from pydrake.all import *
from kinova_station import EndEffectorTarget, GripperTarget

class CustomController(LeafSystem):
    """
    A simple controller for a kinova gen3 robot.

    Inputs:
        - Current end-effector pose
        - Current end-effector twist
        - Current gripper position

    Outputs:
        - Desired end-effector twist
        - Desired gripper velocity
    """
    def __init__(self):              
        LeafSystem.__init__(self)
        self.set_name("custom_controller")

        # Inputs
        self.pose_input = self.DeclareVectorInputPort(
            "ee_pose",
            BasicVector(6))
        self.twist_input = self.DeclareVectorInputPort(
            "ee_twist",
            BasicVector(6))
        self.gripper_input = self.DeclareVectorInputPort(
            "gripper_position",
            BasicVector(1))
        
        # Outputs
        self.DeclareVectorOutputPort(
                "ee_command",
                BasicVector(6),
                self.CalcEndEffectorCommand)
        self.DeclareAbstractOutputPort(
                "ee_command_type",
                lambda: AbstractValue.Make(EndEffectorTarget.kTwist),
                self.SetEndEffectorCommandType)
        self.DeclareVectorOutputPort(
                "gripper_command",
                BasicVector(1),
                self.CalcGripperCommand)
        self.DeclareAbstractOutputPort(
                "gripper_command_type",
                lambda: AbstractValue.Make(GripperTarget.kVelocity),
                self.SetGripperCommandType)

    def SetGripperCommandType(self, context, output):
        command_type = GripperTarget.kVelocity  # always send gripper velocity commands
        output.SetFrom(AbstractValue.Make(command_type))

    def SetEndEffectorCommandType(self, context, output):
        command_type = EndEffectorTarget.kTwist # always send end-effector twist commands
        output.SetFrom(AbstractValue.Make(command_type))

    def CalcGripperCommand(self, context, output):
        gripper_velocity = 0
        output.SetFromVector([gripper_velocity])

    def CalcEndEffectorCommand(self, context, output):

        t = context.get_time()   # we can obtain the current time in this way,
                                 # which is sometimes useful for designing controllers
                                 # that depend on time

        ee_pose = self.pose_input.Eval(context)   # Values from the input ports
        ee_twist = self.twist_input.Eval(context) # can be extracted in this way

        twist_target = np.zeros(6)
        if t < 5:
            twist_target[5] = 0.1
        else:
            twist_target[5] = -0.1

        output.SetFromVector(twist_target)

Note that this controller inherits from `LeafSystem`, which is a special type of system diagram. That allows us to connect our controller up with the `KinovaStation`, as we'll see in the next section.

The main place that you can make modifications to the controller is in the `CalcEndEffectorCommand` method. If you're interested in the other stuff, like how to declare input and output ports, take a look at the Drake documentation. 

Now that we've defined our custom controller class, let's create an instance of it. 

In [42]:
my_controller = CustomController()


Can you make a plot of the controller's input and output ports, like we did for the `KinovaStation` above?

In [ ]:
# Make a plot showing the controller's input and output ports here

# Connecting the Controller

Now that we have our controller, we need to connect it to the system diagram. We do so with the following code:

In [43]:
# First, we create a DiagramBuilder object and add the station and 
# the controller to the diagram. 
builder = DiagramBuilder()
station = builder.AddSystem(station)
controller = builder.AddSystem(my_controller)

# Next, we connect the station's outputs to the controller's inputs
builder.Connect(
    station.GetOutputPort("measured_ee_pose"),
    controller.GetInputPort("ee_pose"))
builder.Connect(
    station.GetOutputPort("measured_ee_twist"),
    controller.GetInputPort("ee_twist"))
builder.Connect(
    station.GetOutputPort("measured_gripper_position"),
    controller.GetInputPort("gripper_position"))

# Then, we connect the controller's outputs to the station's inputs
builder.Connect(
    controller.GetOutputPort("ee_command"),
    station.GetInputPort("ee_target"))
builder.Connect(
    controller.GetOutputPort("ee_command_type"),
    station.GetInputPort("ee_target_type"))
builder.Connect(
    controller.GetOutputPort("gripper_command"),
    station.GetInputPort("gripper_target"))
builder.Connect(
    controller.GetOutputPort("gripper_command_type"),
    station.GetInputPort("gripper_target_type"))

# And finally, compile the overall system diagram
diagram = builder.Build()

To check that we've connected everything correctly, we can make a quick plot of the overall system diagram.

In [ ]:
plot_system_graphviz(diagram, max_depth=1)
plt.show()

If that looks good, we can initialize and run a quick simulation:

In [ ]:
diagram_context = diagram.CreateDefaultContext()

station.go_home(diagram, diagram_context, name="Home")
station.SetManipulandStartPositions(diagram, diagram_context)

simulator = Simulator(diagram, diagram_context)
simulator.set_target_realtime_rate(1.0)
simulator.set_publish_every_time_step(False)

simulator.Initialize()
simulator.AdvanceTo(10.0)

# Challenges

Easy:
- What are the units for the joint velocity?
- What is the difference between a pose and a position? Velocity and Twist?

Medium:
- What does the current controller do?
- Change the controller so that the end-effector is just above the ground after 10 seconds. 
- Change the controller so that the end-effector moves in a circle
- Modify the controller so that the gripper opens and closes several times
- Try sending position targets rather than velocity targets to the gripper

Hard:
- Why does the end-effector rotate sometimes, even if we are sending twist commands with zero angular velocity
    - Hint: it's related to the fact that we are doing open-loop control
    - Change the controller to a closed-loop PD controller
- Modify the custom controller so that you pick up the peg.
- Try changing the end-effector command type. Which is your favorite?